<a href="https://colab.research.google.com/github/Olyvver-bit/Softserve/blob/main/SoftServe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import time
import sys

# Exits program
def exit_program():
  sys.exit(0)

# Function to take in a public IP address code (user can find at "https://api.ipify.org//") and return the latitude and longitude of the provided IP
# (API used: https://ip-api.com/docs/api:json)
def get_loc_chars(loc):
  res_loc = requests.get("http://ip-api.com/json/" + loc + "?fields=lat,lon")

  # Handle rate limit when there are too many rate requests
  if res_loc.status_code == 429:
    print("Rate limit reached. Retrying in 60 seconds...")
    time.sleep(60) # Wait for 60 seconds and retry
    res_loc = requests.get("http://ip-api.com/json/" + loc + "?fields=lat,lon")

  # Return response date if successful, otherwise exits program
  elif res_loc.ok:
    return res_loc.json()
  else:
    print(f"Error: {res_loc.status_code}")
    exit_program()
    return None

# Prompts user to input IP to program
print("Are you ready for a soft serve?")
print("Please input the IP address code here to determine your location: https://api.ipify.org//")
geo_info = get_loc_chars(input())

# Defines vars latitude and longitude retrieved from dictionary object created from accessing IP API
latitude = geo_info.get("lat")
longitude = geo_info.get("lon")

# Function to return forecast information for the next 7 days gathered from the National Weather Service at a 2.5 square kilometer surrounding a given latitude and longitude
# (API used: https://www.weather.gov/documentation/services-web-api)
def get_weather_info(lat,lon):
  res_weather = requests.get("https://api.weather.gov/points/" + str(lat) + "," + str(lon))

  # Handle rate limit when there are too many rate requests
  if res_weather.status_code == 429:
    print("Rate limit reached. Retrying in 60 seconds...")
    time.sleep(60) # Wait for 60 seconds and retry
    res_weather = requests.get("https://api.weather.gov/points/" + str(lat) + "," + str(lon))

  # Return response date if successful, otherwise exits program
  elif res_weather.ok:
    res_weather.json()
    return res_weather.json()
  else:
    print('Error:', res_weather.status_code)
    print ('IP address invalid, please make sure IP is public, IPv4/IPv6, and located in the US')
    exit_program()
    return None

# Define var weather_info as instance of get_weather_info method using the specific user's latitude and longitude
weather_info = get_weather_info(latitude,longitude)

# Filters broad forecast data to the hourly forecast for the next 7 days
def get_hourly_forecast(broad_forecast):
  hourly_forecast = requests.get(broad_forecast.get('properties').get('forecastHourly'))

  if hourly_forecast.ok:
          hourly_forecast.json()
          return hourly_forecast.json()
  else:
          print('Error:', hourly_forecast.status_code)
          print('No hourly forecast found at your location please make sure you are located in the US')
          exit_program()
          return None

# Define var hourly_forecast as instance of get_hourly_forecast using var weather_info as input
hourly_forecast = get_hourly_forecast(weather_info)


# Define var hourly_forecast_list using get() functions to retrieve list of forecast information per hour for the next 7 days
hourly_forecast_list = hourly_forecast.get('properties').get('periods')

# Gets temperature at a specific hour in the next 7 days where x represents x hours after current time
def get_temp(x):
  return hourly_forecast_list[x-1].get('temperature')

# Determines if the weather is suitable for soft serve (if the temperature in the next hour is greater than 78 then you are encouraged to get some)
def ice_cream():
  temp = (get_temp(1))
  if (temp > 78):
    return print(f"The sun's out and it's a perfect {temp}°F — ideal for a soft serve treat! Go grab one and cool off!")
  else:
    return print(f"Brrr, it's a chilly {temp}°F outside! Maybe save the ice cream for a warmer day and grab something cozy instead!")

ice_cream()

Are you ready for a soft serve?
Please input the IP address code here to determine your location: https://api.ipify.org//
